### GitHub repository
Link to repository used to colaborate on the assignment:
https://github.com/KarolineKlan/Assignments_ComSocSci2024.git

### Contribution statement

Team members:

- Jacob (s214596)
- Kristoffer ()
- Karoline (s214638)

All members collaborated and contributed to every part of the assignment.


# Assignment 1
## Part 1 - Webscraping
In the following task we use web-scraping tools to get the list of participants in the International Conference in Computational Social Science 2023

In [ ]:
import pandas

## Part 2 - Ready Made vs Custom Made Data

Write

## Part 3 - Gathering Research Articles using the OpenAlex API

## Part 4: The Network of Computational Social Scientists

In this part of the assignment we construct and investigate the Computational Social Scientists Network.